# Original SPICE

In [ ]:
from __future__ import print_function, division
import os

import torch
import sys
sys.path.insert(0, './')

from fixmatch.utils import net_builder
from fixmatch.datasets.ssl_dataset_robust import SSL_Dataset
from fixmatch.datasets.data_utils import get_data_loader
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def calculate_acc(ypred, y, return_idx=False):
    """
    Calculating the clustering accuracy. The predicted result must have the same number of clusters as the ground truth.

    ypred: 1-D numpy vector, predicted labels
    y: 1-D numpy vector, ground truth
    The problem of finding the best permutation to calculate the clustering accuracy is a linear assignment problem.
    This function construct a N-by-N cost matrix, then pass it to scipy.optimize.linear_sum_assignment to solve the assignment problem.

    """
    assert len(y) > 0
    assert len(np.unique(ypred)) == len(np.unique(y))

    s = np.unique(ypred)
    t = np.unique(y)

    N = len(np.unique(ypred))
    C = np.zeros((N, N), dtype=np.int32)
    for i in range(N):
        for j in range(N):
            idx = np.logical_and(ypred == s[i], y == t[j])
            C[i][j] = np.count_nonzero(idx)

    # convert the C matrix to the 'true' cost
    Cmax = np.amax(C)
    C = Cmax - C
    row, col = linear_sum_assignment(C)
    # calculating the accuracy according to the optimal assignment
    count = 0
    for i in range(N):
        idx = np.logical_and(ypred == s[row[i]], y == t[col[i]])
        count += np.count_nonzero(idx)

    if return_idx:
        return 1.0 * count / len(y), row, col
    else:
        return 1.0 * count / len(y)


def calculate_nmi(predict_labels, true_labels):
    # NMI
    nmi = metrics.normalized_mutual_info_score(true_labels, predict_labels, average_method='geometric')
    return nmi


def calculate_ari(predict_labels, true_labels):
    # ARI
    ari = metrics.adjusted_rand_score(true_labels, predict_labels)
    return ari

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--load_path', type=str, default='./models/model_stl10.pth')
parser.add_argument('--scores_path', type=str, default=None)
parser.add_argument('--use_train_model', action='store_true')

'''
Backbone Net Configurations
'''
parser.add_argument('--net', type=str, default='WideResNet_stl10')
parser.add_argument('--net_from_name', type=bool, default=False)
parser.add_argument('--depth', type=int, default=28)
parser.add_argument('--widen_factor', type=int, default=2)
parser.add_argument('--leaky_slope', type=float, default=0.1)
parser.add_argument('--dropout', type=float, default=0.0)

'''
Data Configurations
'''
parser.add_argument('--batch_size', type=int, default=256)
parser.add_argument('--data_dir', type=str, default='./datasets/stl10')
parser.add_argument('--dataset', type=str, default='stl10')
parser.add_argument('--num_classes', type=int, default=10)
parser.add_argument('--label_file', type=str, default=None)
parser.add_argument('--all', type=int, default=0)
parser.add_argument('--unlabeled', type=bool, default=False)
args = parser.parse_args([])

In [ ]:
checkpoint_path = os.path.join(args.load_path)
checkpoint = torch.load(checkpoint_path)
load_model = checkpoint['train_model'] if args.use_train_model else checkpoint['eval_model']

In [ ]:
for k in list(load_model.keys()):

        # Initialize the feature module with encoder_q of moco.
        if k.startswith('model.'):
            # remove prefix
            load_model[k[len('model.'):]] = load_model[k]

            del load_model[k]
            # print(k)

if args.net in ['WideResNet', 'WideResNet_stl10', 'WideResNet_tiny', 'resnet18', 'resnet18_cifar', 'resnet34']:
    _net_builder = net_builder(args.net,
                               args.net_from_name,
                               {'depth': args.depth,
                                'widen_factor': args.widen_factor,
                                'leaky_slope': args.leaky_slope,
                                'dropRate': args.dropout})
elif args.net == 'ClusterResNet':
    _net_builder = net_builder(args.net,
                               args.net_from_name,
                               {'input_size': args.input_size})
else:
    raise TypeError

In [ ]:
net = _net_builder(num_classes=args.num_classes)
net.load_state_dict(load_model)
if torch.cuda.is_available():
    net.cuda()
net.eval()

In [ ]:
_eval_dset = SSL_Dataset(name=args.dataset, train=False, data_dir=args.data_dir, label_file=None, all=args.all, unlabeled=False)
# print(args.all)

eval_dset = _eval_dset.get_dset()
print(len(eval_dset))

In [ ]:
eval_loader = get_data_loader(eval_dset,
                              args.batch_size, 
                              num_workers=1)

In [ ]:
acc = 0.0
labels_pred = []
labels_gt = []
scores = []
X = []
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        logit = net(image)

        scores.append(logit.cpu().numpy())

        labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt.append(target.cpu().numpy())
        
        X.append(image.cpu().numpy())

scores = np.concatenate(scores, axis=0)
labels_pred = np.concatenate(labels_pred, axis=0)
labels_gt = np.concatenate(labels_gt, axis=0)
X = np.concatenate(X, axis=0)

In [ ]:
try:
    acc = calculate_acc(labels_pred, labels_gt)
except:
    acc = -1

In [ ]:
nmi = calculate_nmi(labels_pred, labels_gt)
ari = calculate_ari(labels_pred, labels_gt)

print(f"Test Accuracy: {acc}, NMI: {nmi}, ARI: {ari}")

## GAN Attack

In [ ]:
import importlib
torch.cuda.empty_cache()

In [ ]:
import gan_attack

In [ ]:
use_cuda=True
image_nc=3
epochs = 60
batch_size = 256
BOX_MIN = 0
BOX_MAX = 1
model_num_labels = 10
device = 'cuda'

In [ ]:
GAN = gan_attack.GAN_Attack(device, net, model_num_labels, image_nc, BOX_MIN, BOX_MAX, '256stl10')

In [ ]:
import sys

# Holding the original output object. i.e. console out
orig_stdout = sys.stdout

# Opening the file to write file deletion logs.
f = open('outgan_train256_s10.txt', 'a+')

# Changing standard out to file out. 
sys.stdout = f
# This will write to the file. 
print("xyz") 

In [ ]:
# %%time
advGAN.train(eval_loader, 600)

In [ ]:
# Closing the file.
f.close()

# replacing the original output format to stdout.
sys.stdout = orig_stdout

# This will print onto the console.
print("xyz") 

## Testing attack

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_stl10_epoch_600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
def save_examples(images, labels, noise=False, bno=0, adv=False, orig=False):
    print(type(images[0]), type(labels))
    MEAN = torch.tensor([0.4914, 0.4822, 0.4465]).cuda()
    STD = torch.tensor([0.2023, 0.1994, 0.2010]).cuda()
    
    for i in range(min(len(images), 20)):
        img = images[i]
        img = img * STD[:, None, None] + MEAN[:, None, None]
        npimg = img.detach().cpu().numpy()   # convert from tensor
        npimg = np.clip(npimg, 0, 1)
        if orig:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'Images/S10/SPICE/orig/SPICE_s10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if adv:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'Images/S10/SPICE/adv/SPICE_s10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if noise:
            npimg = npimg / 2 + 0.5 
            plt.imsave(f'Images/S10/SPICE/noise/SPICE_s10_b{bno}_{i}_noise_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
    
    

In [ ]:
acc_adv = 0.0
labels_pred_adv = []
labels_gt_adv = []
scores = []
X_adv = []
step = 0
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        perturbation = pretrained_G(image)
        perturbation = torch.clamp(perturbation, -0.2, 0.2)        
        adv_imgs = perturbation + image
        
        logit = net(adv_imgs)

        scores.append(logit.cpu().numpy())

        labels_pred_adv.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt_adv.append(target.cpu().numpy())
        X_adv.append(adv_imgs.cpu().numpy())
        save_examples(image, labels_gt_adv[-1], bno=step, orig=True)
        save_examples(adv_imgs, labels_pred_adv[-1], bno=step, adv=True)
        save_examples(perturbation, labels_gt_adv[-1], bno=step, noise=True)
        step +=1
#         pps(image, adv_imgs, target.cpu().numpy(), torch.max(logit, dim=-1)[1].cpu().numpy())
#         break
#         pp(image, labels_gt)

scores = np.concatenate(scores, axis=0)
labels_pred_adv = np.concatenate(labels_pred_adv, axis=0)
labels_gt_adv = np.concatenate(labels_gt_adv, axis=0)
X_adv = np.concatenate(X, axis=0)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
from scipy.misc import toimage

MEAN = torch.tensor([0.4914, 0.4822, 0.4465]).cuda()
STD = torch.tensor([0.2023, 0.1994, 0.2010]).cuda()

# x = normalized_img * STD[:, None, None] + MEAN[:, None, None]


def pp(images, labels):
    MEAN = torch.tensor([0.4914, 0.4822, 0.4465]).cuda()
    STD = torch.tensor([0.229, 0.224, 0.225]).cuda()
    print(type(images[0]), type(labels))
    print(images.shape)
#     w = 
#     h = 10
    fig = plt.figure(figsize=(15, 15))
    columns = 11
    rows = 12
    for i in range(35):
        fig.add_subplot(rows, columns, i+1)
        img = images[i]
#         img = img / 2 + 0.5
#         img = img / 2 + 0.5   # unnormalize
        img = img * STD[:, None, None] + MEAN[:, None, None]
        img = img.detach().cpu().numpy()
        img = np.clip(img, 0, 1)
#         img = (img * 255).astype(np.uint8)
#         img = img / 2 + 0.5
#         img = img / 2 + 0.5 
#         npimg = img.detach().cpu().numpy()   # convert from tensor
        
#         plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
        plt.imshow(img.transpose(1, 2, 0))
#         plt.imshow(npimg)
#         plt.title('#{}: {}'.format(i, labels[i]))
    plt.show()

count = 0
def pps(images, adv_imgs, orig, predicted):
    global count
    MEAN = torch.tensor([0.4914, 0.4822, 0.4465]).cuda()
    STD = torch.tensor([0.229, 0.224, 0.225]).cuda()
    
    print(images.shape)

    for i, _ in enumerate(images):
        img = images[i]
        img_a = adv_imgs[i]

        img = img * STD[:, None, None] + MEAN[:, None, None]
        img = img.detach().cpu().numpy()
        img = np.clip(img, 0, 1)
        
        img_a = img_a * STD[:, None, None] + MEAN[:, None, None]
        img_a = img_a.detach().cpu().numpy()
        img_a = np.clip(img_a, 0, 1)

        
#         plt.show(img.transpose(1, 2, 0))
        plt.imsave(f'outputs/stl10/orig/{count}_{orig[i]}.png', img.transpose(1, 2, 0))
        plt.imsave(f'outputs/stl10/adv/{count}_{predicted[i]}.png', img_a.transpose(1, 2, 0))
        count += 1
        
#         return 
#     plt.show()

In [ ]:
acc_adv = 0.0
labels_pred_adv = []
labels_gt_adv = []
scores = []
X_adv = []
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        perturbation = pretrained_G(image)
        perturbation = torch.clamp(perturbation, -0.2, 0.2)        
        adv_imgs = perturbation + image
        
        logit = net(adv_imgs)

        scores.append(logit.cpu().numpy())

        labels_pred_adv.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt_adv.append(target.cpu().numpy())
        X_adv.append(adv_imgs.cpu().numpy())
#         pps(image, adv_imgs, target.cpu().numpy(), torch.max(logit, dim=-1)[1].cpu().numpy())
#         break
#         pp(image, labels_gt)

scores = np.concatenate(scores, axis=0)
labels_pred_adv = np.concatenate(labels_pred_adv, axis=0)
labels_gt_adv = np.concatenate(labels_gt_adv, axis=0)
X_adv = np.concatenate(X, axis=0)

In [ ]:
np.unique(labels_gt)

In [ ]:
try:
    acc_adv = calculate_acc(labels_pred_adv, labels_gt_adv)
except:
    acc = -1

In [ ]:
nmi = calculate_nmi(labels_pred_adv, labels_gt_adv)
ari = calculate_ari(labels_pred_adv, labels_gt_adv)

print(f"Test Accuracy: {acc_adv}, NMI: {nmi}, ARI: {ari}")

In [ ]:
# for image, target, _ in eval_loader:
#     targets, inputs = target.to(device), image.to(device)
#     perturbation = pretrained_G(inputs)
#     perturbation = torch.clamp(perturbation, -0.2, 0.2)
#     adv_imgs = perturbation + inputs
# #     print(inputs[0])
#     pp(inputs, targets)
#     pp(adv_imgs, targets)
#     pp(perturbation, targets)
#     break

# Additional Experirments for TSNE/PCA

In [ ]:
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X.shape

In [ ]:
X2.shape

In [ ]:
X2 = X.copy()
X2_adv = X_adv.copy()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3)
X2 = X2.reshape(8000, 3*96*96)
X2 = X2.T

pca_adv = PCA(n_components=3)

X2_adv = X2.reshape(8000, 3*96*96)
X2_adv = X2_adv.T

In [ ]:
pca.fit(X2)
pca_x = pca.components_

pca_adv.fit(X2_adv)
pca_adv_x = pca_adv.components_

In [ ]:
pca_x.shape

In [ ]:
pca_adv_x.shape

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import seaborn as sns
class2colors = sns.color_palette("tab10", 10) # 10 colors

class2markers = ["+", "x", "*", "P", "o", "^", "d", "p", "s", "8"]

In [ ]:
def check3dboundary(x,y,z):
    return -0.04 <= min(x,y,z) and max(x,y,z) <= 0.04

def check2dboundary(x,y):
    return -0.04 <= min(x,y) and max(x,y) <= 0.04

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerBase

list_color  = class2colors[:]
list_mak    =  ["o"]*10
lis = ["plane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"]

list_lab    = [j[0].upper() + j[1:] for j in lis]


list_color2 = ['tab:red', 'tab:blue']
list_mak2 = ['o', 'o']
list_lab2  = ['Adversarial', 'Original']

# ax = plt.gca()

class MarkerHandler(HandlerBase):
    def create_artists(self, legend, tup,xdescent, ydescent,
                        width, height, fontsize,trans):
        return [plt.Line2D([width/2], [height/2.],ls="",
                       marker=tup[1],color=tup[0], transform=trans)]


# ax.legend(list(zip(list_color,list_mak)), list_lab, handler_map={tuple:MarkerHandler()}) 

# plt.show()

In [ ]:
ax = plt.axes(projection='3d')
# ax.scatter3D(pca_adv_x[0], pca_adv_x[1], pca_adv_x[2], color='red', marker='^')

# gt -> marker; predicted -> color
for i in range(80):
#     for clas in range(10):
#         if labels_pred[i] == clas:
#     ax.scatter3D(pca_x[0, i], pca_x[1, i], pca_x[2, i], color=class2colors[labels_pred[i]], marker=class2markers[labels_gt[i]], label=labels_gt, edgecolor='black')
    ax.scatter3D(pca_x[0, i], pca_x[1, i], pca_x[2, i], color=class2colors[labels_pred[i]], edgecolor='black')
ax.set_zlim(-0.04, 0.04)
ax.set_xlim(-0.04, 0.04)
ax.set_ylim(-0.04, 0.04)
# ax.xticks(color='w')
# ax.yticks(color='w')
# ax.zticks(color='w')
# ax.set_xticks([])
# ax.set(ylabel=None)
# ax.set(xlabel=None)
# ax.tick_params(axis='x', colors='white')
# ax.tick_params(axis='y', colors='white')
# ax.tick_params(axis='z', colors='white')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
# plt.legend(class2colors, [0]*10)
ax.legend(list(zip(list_color,list_mak)), list_lab, handler_map={tuple:MarkerHandler()}, fontsize = 8) 
plt.savefig('3d_orig.png')

    # plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))
# plt.show()

In [ ]:
ax = plt.axes(projection='3d')
# ax.scatter3D(pca_adv_x[0], pca_adv_x[1], pca_adv_x[2], color='red', marker='^')

plt.ylim((-0.04, 0.04))
plt.xlim((-0.04, 0.04))
# gt -> marker; predicted -> color
for i in range(8000):
#     for clas in range(10):
#         if labels_pred_adv[i] == clas:
    if check3dboundary(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i]):
        ax.scatter3D(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i], color=class2colors[labels_pred_adv[i]], marker=class2markers[labels_gt_adv[i]], edgecolor='black')

ax.set_zlim(-0.04, 0.04)
ax.set_xlim(-0.04, 0.04)
ax.set_ylim(-0.04, 0.04)

# plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))

In [ ]:
ax = plt.axes(projection='3d')
# ax.scatter3D(pca_adv_x[0], pca_adv_x[1], pca_adv_x[2], color='red', marker='^')

# plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))
# # plt.zlim((-0.04, 0.04))

# gt -> color; predicted -> marker
# ax.set_zlim(-0.04, 0.04)
# ax.set_xlim(-0.04, 0.04)
# ax.set_ylim(-0.04, 0.04)

for i in range(8000):
#     for clas in range(10):
#         if labels_pred_adv[i] == clas:
    ax.scatter3D(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i], color=class2colors[labels_gt_adv[i]], marker=class2markers[labels_pred_adv[i]], edgecolor='black')

ax.set_zlim(-0.04, 0.04)
ax.set_xlim(-0.04, 0.04)
ax.set_ylim(-0.04, 0.04)

# plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))

In [ ]:
# in
ax = plt.axes(projection='3d')
# ax.scatter3D(pca_adv_x[0], pca_adv_x[1], pca_adv_x[2], color='red', marker='^')

# gt -> color; predicted -> marker
for i in range(8000):
#     for clas in range(10):
#         if labels_pred[i] == clas:
    ax.scatter3D(pca_x[0, i], pca_x[1, i], pca_x[2, i], color=class2colors[labels_gt[i]], edgecolor='black')

ax.set_zlim(-0.04, 0.04)
ax.set_xlim(-0.04, 0.04)
ax.set_ylim(-0.04, 0.04)

ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
# plt.legend(class2colors, [0]*10)
ax.legend(list(zip(list_color,list_mak)), list_lab, handler_map={tuple:MarkerHandler()}, loc='upper right', bbox_to_anchor=(1.2, 1.05)) 
plt.savefig('3d_orig.png', dpi=600, bbox_inches='tight')
    # plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))
# plt.show()

In [ ]:
ax = plt.axes(projection='3d')
# ax.scatter3D(pca_adv_x[0], pca_adv_x[1], pca_adv_x[2], color='red', marker='^')

# bbox uinches = tight, dpi 600, 

# plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))
# # plt.zlim((-0.04, 0.04))

# gt -> color; predicted -> marker
# ax.set_zlim(-0.04, 0.04)
# ax.set_xlim(-0.04, 0.04)  
# ax.set_ylim(-0.04, 0.04)

for i in range(8000):
#     for clas in range(10):
#         if labels_pred_adv[i] == clas:
#     if -0.04 <= min(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i]) and max(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i]) <= 0.04: 
    if check3dboundary(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i]):
        ax.scatter3D(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i], color=class2colors[labels_gt_adv[i]], edgecolor='black')

ax.set_zlim(-0.04, 0.04)
ax.set_xlim(-0.04, 0.04)
ax.set_ylim(-0.04, 0.04)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])

# ax.legend(list(zip(list_color,list_mak)), list_lab, handler_map={tuple:MarkerHandler()}, loc='upper right', bbox_to_anchor=(1.2, 1.05)) 
# ax.legend(list(zip(list_color,list_mak)), list_lab, handler_map={tuple:MarkerHandler()}, fontsize=9, handletextpad=0)
plt.savefig('3d_adv.png', dpi=600, bbox_inches='tight')

# plt.legend(class2colors, [0]*10)
# plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))

In [ ]:
ax = plt.axes(projection='3d')
# ax.scatter3D(pca_adv_x[0], pca_adv_x[1], pca_adv_x[2], color='red', marker='^')

# plt.ylim((-0.04, 0.04))
# plt.xlim((-0.04, 0.04))
# # plt.zlim((-0.04, 0.04))

# gt -> color; predicted -> marker
# ax.set_zlim(-0.04, 0.04)
# ax.set_xlim(-0.04, 0.04)
# ax.set_ylim(-0.04, 0.04)

for i in range(8000):
#     for clas in range(10):
#         if labels_pred_adv[i] == clas:
#     if -0.04 <= min(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i]) and max(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i]) <= 0.04: 
    if check3dboundary(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i]):
        ax.scatter3D(pca_x[0, i], pca_x[1, i], pca_x[2, i], color="tab:blue", marker='o')
        ax.scatter3D(pca_adv_x[0, i], pca_adv_x[1, i], pca_adv_x[2, i], color="tab:red", marker='o')

ax.set_zlim(-0.04, 0.04)
ax.set_xlim(-0.04, 0.04)
ax.set_ylim(-0.04, 0.04)
ax.set_xticklabels(['1'], color='w')
ax.set_yticklabels(['1'], color='w')
ax.set_zticklabels(['1'], color='w')
ax.tick_params(False)
# plt.legend(class2colors, [0]*10)
ax.legend(list(zip(list_color2,list_mak2)), list_lab2, handler_map={tuple:MarkerHandler()}, loc='upper center', fontsize=15, ncol=2) 
plt.tight_layout()
plt.savefig('3d_both.pdf', dpi=400, bbox_inches='tight')

In [ ]:
X2 = X.copy()
X2_adv = X_adv.copy()

pca = PCA(n_components=2)
X2 = X2.reshape(8000, 3*96*96)
X2 = X2.T

pca_adv = PCA(n_components=2)

X2_adv = X2.reshape(8000, 3*96*96)
X2_adv = X2_adv.T

pca.fit(X2)
pca_x = pca.components_

pca_adv.fit(X2_adv)
pca_adv_x = pca_adv.components_


In [ ]:
# plt.scatter(pca_x[0], pca_x[1], color="red", marker='^', edgecolor='black')
# plt.scatter(pca_adv_x[0], pca_adv_x[1], color="blue", marker='o', edgecolor='black')
# plt.show()

#markers are predicted labels, and orig is blue and adv is red.
for i in range(8000):
    plt.scatter(pca_x[0, i], pca_x[1, i], color="tab:blue", marker='o', edgecolor='black')
    plt.scatter(pca_adv_x[0, i], pca_adv_x[1, i], color="tab:red",marker='o', edgecolor='black')

plt.ylim((-0.04, 0.04))
plt.xlim((-0.04, 0.04))
plt.legend()
plt.show()

In [ ]:
# plt.plot(pca_x[0], pca_x[1])
# ax = plt.axes(projection='3d')
# ax.scatter3D(pca_x[0], pca_x[1], pca_x[2]);

In [ ]:
# # %matplotlib notebook
# # plt.plot(pca_x[0], pca_x[1])
# ax = plt.axes(projection='3d')
# ax.scatter3D(pca_adv_x[0], pca_adv_x[1], pca_adv_x[2], color='red', marker='^')
# ax.scatter3D(pca_x[0], pca_x[1], pca_x[2], color='blue', marker='o')

In [ ]:
# plt.scatter(pca_x[0], pca_x[1], color="red", marker='^', edgecolor='black')
# plt.scatter(pca_adv_x[0], pca_adv_x[1], color="blue", marker='o', edgecolor='black')
# plt.show()

In [ ]:
for i in range(8000):
    for clas in range(10):
        if labels_pred_adv[i] == clas:
            plt.scatter(pca_adv_x[0, i], pca_adv_x[1, i], color=color_l[clas], marker='o', edgecolor='black')
plt.ylim((-0.04, 0.04))
plt.xlim((-0.04, 0.04))
plt.show()
#     else:
#         plt.scatter(pca_adv_x[0, i], pca_adv_x[1, i], color="blue", marker='^', edgecolor='black')

In [ ]:
for i in range(8000):
    for clas in range(10):
        if labels_pred[i] == clas:
            plt.scatter(pca_x[0, i], pca_x[1, i], color=color_l[clas], marker='o', edgecolor='black')
plt.ylim((-0.04, 0.04))
plt.xlim((-0.04, 0.04))
plt.show()
#     else:
#         plt.scatter(pca_adv_x[0, i], pca_adv_x[1, i], color="blue", marker='^', edgecolor='black')

# Transferability attack

In [ ]:
pretrained_generator_path = '../Generator_Models/STL10/netG_CC_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

acc_adv = 0.0
labels_pred = []
labels_gt = []
scores = []
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        perturbation = pretrained_G(image)
        perturbation = torch.clamp(perturbation, -0.3, 0.3)        
        adv_imgs = perturbation + image
        
        logit = net(adv_imgs)

        scores.append(logit.cpu().numpy())

        labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt.append(target.cpu().numpy())

scores = np.concatenate(scores, axis=0)
labels_pred = np.concatenate(labels_pred, axis=0)
labels_gt = np.concatenate(labels_gt, axis=0)

try:
    acc_adv = calculate_acc(labels_pred, labels_gt)
except:
    acc = -1
    
nmi = calculate_nmi(labels_pred, labels_gt)
ari = calculate_ari(labels_pred, labels_gt)

print(f"Test Accuracy: {acc_adv}, NMI: {nmi}, ARI: {ari}")

for image, target, _ in eval_loader:
    targets, inputs = target.to(device), image.to(device)
    perturbation = pretrained_G(inputs)
    perturbation = torch.clamp(perturbation, -0.3, 0.3)
    adv_imgs = perturbation + inputs
#     print(inputs[0])
    pp(inputs, targets)
    pp(adv_imgs, targets)
    pp(perturbation, targets)
    break

In [ ]:
pretrained_generator_path = '../Generator_Models/STL10/netG_MICE_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

acc_adv = 0.0
labels_pred = []
labels_gt = []
scores = []
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        perturbation = pretrained_G(image)
        perturbation = torch.clamp(perturbation, -0.5, 0.5)        
        adv_imgs = perturbation + image
        
        logit = net(adv_imgs)

        scores.append(logit.cpu().numpy())

        labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt.append(target.cpu().numpy())

# print(perturbation)
scores = np.concatenate(scores, axis=0)
labels_pred = np.concatenate(labels_pred, axis=0)
labels_gt = np.concatenate(labels_gt, axis=0)

try:
    acc_adv = calculate_acc(labels_pred, labels_gt)
    
except:
    acc = -1
    
nmi = calculate_nmi(labels_pred, labels_gt)
ari = calculate_ari(labels_pred, labels_gt)

print(f"Test Accuracy: {acc_adv}, NMI: {nmi}, ARI: {ari}")

for image, target, _ in eval_loader:
    targets, inputs = target.to(device), image.to(device)
    perturbation = pretrained_G(inputs)
#     perturbation = torch.clamp(perturbation, -0.5, 0.5)
    adv_imgs = perturbation + inputs
#     print(inputs[0])
    pp(inputs, targets)
    pp(adv_imgs, targets)
    pp(perturbation, targets)
    break

In [ ]:

pretrained_generator_path = '../Generator_Models/STL10/netG_SCAN_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

acc_adv = 0.0
labels_pred = []
labels_gt = []
scores = []
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        perturbation = pretrained_G(image)
        perturbation = torch.clamp(perturbation, -0.2, 0.2)        
        adv_imgs = perturbation + image
        
        logit = net(adv_imgs)

        scores.append(logit.cpu().numpy())

        labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt.append(target.cpu().numpy())

scores = np.concatenate(scores, axis=0)
labels_pred = np.concatenate(labels_pred, axis=0)
labels_gt = np.concatenate(labels_gt, axis=0)

try:
    acc_adv = calculate_acc(labels_pred, labels_gt)
except:
    acc = -1
    
nmi = calculate_nmi(labels_pred, labels_gt)
ari = calculate_ari(labels_pred, labels_gt)

print(f"Test Accuracy: {acc_adv}, NMI: {nmi}, ARI: {ari}")

for image, target, _ in eval_loader:
    targets, inputs = target.to(device), image.to(device)
    perturbation = pretrained_G(inputs)
    perturbation = torch.clamp(perturbation, -0.2, 0.2)
    adv_imgs = perturbation + inputs
#     print(inputs[0])
    pp(inputs, targets)
    pp(adv_imgs, targets)
    pp(perturbation, targets)
    break

In [ ]:
pretrained_generator_path = '../Generator_Models/STL10/netG_RUC_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

acc_adv = 0.0
labels_pred = []
labels_gt = []
scores = []
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        perturbation = pretrained_G(image)
        perturbation = torch.clamp(perturbation, -0.2, 0.2)        
        adv_imgs = perturbation + image
        
        logit = net(adv_imgs)

        scores.append(logit.cpu().numpy())

        labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt.append(target.cpu().numpy())

scores = np.concatenate(scores, axis=0)
labels_pred = np.concatenate(labels_pred, axis=0)
labels_gt = np.concatenate(labels_gt, axis=0)

try:
    acc_adv = calculate_acc(labels_pred, labels_gt)
except:
    acc = -1
    
nmi = calculate_nmi(labels_pred, labels_gt)
ari = calculate_ari(labels_pred, labels_gt)

print(f"Test Accuracy: {acc_adv}, NMI: {nmi}, ARI: {ari}")

for image, target, _ in eval_loader:
    targets, inputs = target.to(device), image.to(device)
    perturbation = pretrained_G(inputs)
    perturbation = torch.clamp(perturbation, -0.2, 0.2)
    adv_imgs = perturbation + inputs
#     print(inputs[0])
    pp(inputs, targets)
    pp(adv_imgs, targets)
    pp(perturbation, targets)
    break

In [ ]:
pretrained_generator_path = '../Generator_Models/STL10/netG_NNM_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

acc_adv = 0.0
labels_pred = []
labels_gt = []
scores = []
with torch.no_grad():
    for image, target, _ in eval_loader:
        image = image.type(torch.FloatTensor).cuda()
        perturbation = pretrained_G(image)
        perturbation = torch.clamp(perturbation, -0.2, 0.2)        
        adv_imgs = perturbation + image
        
        logit = net(adv_imgs)

        scores.append(logit.cpu().numpy())

        labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
        labels_gt.append(target.cpu().numpy())

# print(perturbation)
scores = np.concatenate(scores, axis=0)

labels_pred = np.concatenate(labels_pred, axis=0)
labels_gt = np.concatenate(labels_gt, axis=0)

try:
    acc_adv = calculate_acc(labels_pred, labels_gt)
    
except:
    acc = -1
    
nmi = calculate_nmi(labels_pred, labels_gt)
ari = calculate_ari(labels_pred, labels_gt)

print(f"Test Accuracy: {acc_adv}, NMI: {nmi}, ARI: {ari}")

for image, target, _ in eval_loader:
    targets, inputs = target.to(device), image.to(device)
    perturbation = pretrained_G(inputs)
    perturbation = torch.clamp(perturbation, -0.2, 0.2)
    adv_imgs = perturbation + inputs
#     print(inputs[0])
    pp(inputs, targets)
    pp(adv_imgs, targets)
    pp(perturbation, targets)
    break

In [ ]:
# Additional Experimetns

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_stl10_epoch_600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    acc_adv = 0.0
    labels_pred = []
    labels_gt = []
    scores = []
    norm = 0.0
    with torch.no_grad():
        for image, target, _ in eval_loader:
            image = image.type(torch.FloatTensor).cuda()
            perturbation = pretrained_G(image)
            perturbation = torch.clamp(perturbation, -j, j)    
            norm += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()
            adv_imgs = perturbation + image

            logit = net(adv_imgs)

            scores.append(logit.cpu().numpy())

            labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
            labels_gt.append(target.cpu().numpy())

    scores = np.concatenate(scores, axis=0)
    labels_pred = np.concatenate(labels_pred, axis=0)
    labels_gt = np.concatenate(labels_gt, axis=0)

    try:
        acc2 = calculate_acc(labels_pred, labels_gt)
    except:
        acc = -1

    nmi = calculate_nmi(labels_pred, labels_gt)
    ari = calculate_ari(labels_pred, labels_gt)

    print(f'clamp {j} avg norm: {norm/len(eval_loader)}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc2))
    norm_l.append(norm/len(eval_loader))
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc2)

In [ ]:
# plt.plot(x, y, label = "line 1", linestyle="-")
# plt.plot(y, x, label = "line 2", linestyle="--")
# plt.plot(x, np.sin(x), label = "curve 1", linestyle="-.")
# plt.plot(x, np.cos(x), label = "curve 2", linestyle=":")
import matplotlib.pyplot as plt

plt.plot(norm_l, nmi_l, label = "nmi", linestyle="-")
plt.plot(norm_l, ari_l, label = "ari", linestyle="-")
plt.plot(norm_l, acc_l, label = "acc", linestyle="-")
plt.xlabel("Perturbation Norm")
plt.ylabel("Performace")
plt.legend()
plt.grid()
plt.savefig('spice_cifar10.png')
plt.show()

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)

In [ ]:
#same acc formula

In [ ]:
import eval_cus
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    acc_adv = 0.0
    labels_pred = []
    labels_gt = []
    scores = []
    norm = 0.0
    with torch.no_grad():
        for image, target, _ in eval_loader:
            image = image.type(torch.FloatTensor).cuda()
            perturbation = pretrained_G(image)
            perturbation = torch.clamp(perturbation, -j, j)    
            norm += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()
            adv_imgs = perturbation + image
#             adv_imgs = torch.clamp(adv_imgs, -1, 1)
            
            logit = net(adv_imgs)

            scores.append(logit.cpu().numpy())
            
            

            labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
            labels_gt.append(target.cpu().numpy())

    scores = np.concatenate(scores, axis=0)
    labels_pred = np.concatenate(labels_pred, axis=0)
    labels_gt = np.concatenate(labels_gt, axis=0)
    class_names = ['airplane',
                 'bird',
                 'car',
                 'cat',
                 'deer',
                 'dog',
                 'horse',
                 'monkey',
                 'ship',
                 'truck']
    labels_gt = torch.from_numpy(labels_gt).cuda()
    labels_pred = torch.from_numpy(labels_pred).cuda()
    clustering_stats_adv = eval_cus.check(labels_gt, labels_pred, 20, class_names, 
                                        compute_confusion_matrix=True, 
                    confusion_matrix_file=None, cf20=False)
    acc2 = clustering_stats_adv['ACC']
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    
#     try:
#         acc2 = calculate_acc(labels_pred, labels_gt)
#     except:
#         acc = -1

#     nmi = calculate_nmi(labels_pred, labels_gt)
#     ari = calculate_ari(labels_pred, labels_gt)

    print(f'clamp {j} avg norm: {norm/len(eval_loader)}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc2))
    norm_l.append(norm/len(eval_loader))
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc2)

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)

In [ ]:
import importlib

In [ ]:
import eval_cus

importlib.reload(eval_cus)
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
# clamp = [0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
clamp = [0, 0.2, 1]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    acc_adv = 0.0
    labels_pred = []
    labels_gt = []
    scores = []
    norm = 0.0
    with torch.no_grad():
        for image, target, _ in eval_loader:
            image = image.type(torch.FloatTensor).cuda()
            perturbation = pretrained_G(image)
            perturbation = torch.clamp(perturbation, -j, j)    
            norm += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()
            adv_imgs = perturbation + image
#             adv_imgs = torch.clamp(adv_imgs, -1, 1)
            
            logit = net(adv_imgs)

            scores.append(logit.cpu().numpy())
            
            

            labels_pred.append(torch.max(logit, dim=-1)[1].cpu().numpy())
            labels_gt.append(target.cpu().numpy())

    scores = np.concatenate(scores, axis=0)
    labels_pred = np.concatenate(labels_pred, axis=0)
    labels_gt = np.concatenate(labels_gt, axis=0)
    class_names = ['airplane',
                 'bird',
                 'car',
                 'cat',
                 'deer',
                 'dog',
                 'horse',
                 'monkey',
                 'ship',
                 'truck']
    labels_gt = torch.from_numpy(labels_gt).cuda()
    labels_pred = torch.from_numpy(labels_pred).cuda()
    clustering_stats_adv = eval_cus.check(labels_gt, labels_pred, 20, class_names, 
                                        compute_confusion_matrix=True, 
                    confusion_matrix_file=None, cf20=False, output_file2=f'SPICE_s10_{j}_n{norm}.pdf')
    acc2 = clustering_stats_adv['ACC']
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    
#     try:
#         acc2 = calculate_acc(labels_pred, labels_gt)
#     except:
#         acc = -1

#     nmi = calculate_nmi(labels_pred, labels_gt)
#     ari = calculate_ari(labels_pred, labels_gt)

    print(f'clamp {j} avg norm: {norm/len(eval_loader)}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc2))
    norm_l.append(norm/len(eval_loader))
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc2)